In [1]:
using LorentzianSimplexSolver;

    CondaPkg Found dependencies: /Users/xiaoquer/.julia/packages/PythonCall/83z4q/CondaPkg.toml
    CondaPkg Found dependencies: /Users/xiaoquer/.julia/packages/CondaPkg/0UqYV/CondaPkg.toml
    CondaPkg Found dependencies: /Users/xiaoquer/.julia/environments/v1.11/CondaPkg.toml
    CondaPkg Initialising pixi
             │ /Users/xiaoquer/.julia/artifacts/cc02fad3b04cfb688ba4e3d60648ad8c3afe3a2d/bin/pixi
             │ init
             │ --format pixi
             └ /Users/xiaoquer/Library/Mobile Documents/com~apple~CloudDocs/Documents/Work/effective-spinfoam/code/LorentzianSimplexSolver/.CondaPkg
✔ Created /Users/xiaoquer/Library/Mobile Documents/com~apple~CloudDocs/Documents/Work/effective-spinfoam/code/LorentzianSimplexSolver/.CondaPkg/pixi.toml
    CondaPkg Wrote /Users/xiaoquer/Library/Mobile Documents/com~apple~CloudDocs/Documents/Work/effective-spinfoam/code/LorentzianSimplexSolver/.CondaPkg/pixi.toml
             │ [dependencies]
             │ numpy = "*"
             │ sympy

In [2]:
# ------------------------------------------------------------
# 1. Precision choice (user-controlled)
# ------------------------------------------------------------
const ScalarT = Float64
# const ScalarT = BigFloat

if ScalarT === BigFloat
    LorentzianSimplexSolver.PrecisionUtils.set_big_precision!(prec)
    LorentzianSimplexSolver.PrecisionUtils.set_tolerance!(sqrt(eps(BigFloat)))
else
    LorentzianSimplexSolver.PrecisionUtils.set_tolerance!(1e-10)
end

# ------------------------------------------------------------
# 2. Read simplices
# ------------------------------------------------------------
simplices = [[1,2,3,4,5], [1,2,4,5,6]]

ns = length(simplices)

all_vertices = unique(Iterators.flatten(simplices))
sort!(all_vertices)

Nverts = length(all_vertices)

# ------------------------------------------------------------
# 3. Read vertex coordinates
# ------------------------------------------------------------
vertex_coords = Dict{Int, Vector{ScalarT}}()    

coords_lines = [
    "0, 0, 0, 0",
    "0, 0, 0, 1",
    "0, 0, 1, 1",
    "0, 1, 1, 1",
    "1//2, 1, 1, 1",
    "0, 1, 0, 1",
]

for (v, line) in zip(all_vertices, coords_lines)
    vertex_coords[v] = LorentzianSimplexSolver.PrecisionUtils.parse_numeric_line(line, ScalarT)
end

In [3]:
# ------------------------------------------------------------
# 4. Build geometry
# ------------------------------------------------------------
datasets = LorentzianSimplexSolver.GeometryTypes.GeometryDataset{ScalarT}[]

for (s, simplex) in enumerate(simplices)
    println("\n--- Processing simplex $s with vertices $simplex ---")
    bdypoints = [vertex_coords[v] for v in simplex]
    ds = LorentzianSimplexSolver.GeometryPipeline.run_geometry_pipeline(bdypoints)
    push!(datasets, ds)
end

geom_base = LorentzianSimplexSolver.GeometryTypes.GeometryCollection(datasets);


--- Processing simplex 1 with vertices [1, 2, 3, 4, 5] ---

--- Processing simplex 2 with vertices [1, 2, 4, 5, 6] ---


In [4]:
# ------------------------------------------------------------
# 5. Consistency checks for each simplex
# ------------------------------------------------------------
for (idx, simplex) in enumerate(geom_base.simplex)
    LorentzianSimplexSolver.GeometryConsistency.check_sl2c_parallel_transport(simplex.solgsl2c, simplex.bdybivec55)
    LorentzianSimplexSolver.GeometryConsistency.check_so13_parallel_transport(simplex.solgso13, simplex.bdybivec4d55)
    LorentzianSimplexSolver.GeometryConsistency.check_closure_bivectors(simplex.kappa, simplex.areas, simplex.bdybivec55)
end

✓ SL(2,C) parallel transport satisfied (max residual = 2.7755575615628914e-16).
✓ SO(1,3) parallel transport satisfied (max residual = 5.266250202865052e-16)
✓ Closure condition satisfied for bivectors (max residual = 8.777083671441753e-17)
✓ SL(2,C) parallel transport satisfied (max residual = 2.7755575615628914e-16).
✓ SO(1,3) parallel transport satisfied (max residual = 5.266250202865052e-16)
✓ Closure condition satisfied for bivectors (max residual = 8.777083671441753e-17)


In [5]:
# ------------------------------------------------------------
# 6. Connect simplices + face matching + gauge fixing
# ------------------------------------------------------------
if ns > 1
    println("\nFixing global κ-sign orientation ...")
    LorentzianSimplexSolver.KappaOrientation.fix_kappa_signs!(simplices, geom_base)

    println("\nBuilding global connectivity ...")
    conn = LorentzianSimplexSolver.FourSimplexConnectivity.build_global_connectivity(simplices, geom_base)
    push!(geom_base.connectivity, conn)

    geom_ref    = deepcopy(geom_base)
    geom_parity = deepcopy(geom_base)
    LorentzianSimplexSolver.FaceXiMatching.run_face_xi_matching(geom_ref; sector=:ref)
    LorentzianSimplexSolver.FaceXiMatching.run_face_xi_matching(geom_parity; sector=:parity)
    println("Global connectivity constructed for both relevant orientations.")

    # println("\nRunning final face-matching checks ...")
    LorentzianSimplexSolver.FaceMatchingChecks.check_all(geom_ref)
    LorentzianSimplexSolver.FaceMatchingChecks.check_all(geom_parity)

    println("\nPerform SU(2) and SU(1,1) gauge fixing ...")
    LorentzianSimplexSolver.GaugeFixingSU.run_su2_su11_gauge_fix(geom_ref)
    LorentzianSimplexSolver.GaugeFixingSU.run_su2_su11_gauge_fix(geom_parity)
    println("\nGauge fixing finished.")
end


Fixing global κ-sign orientation ...
[1, 2]

Building global connectivity ...
  SL(2,C) matrices updated:      ✓
  SL(2,C) parity matrices updated: ✓
  Boundary bivectors updated:    ✓
  boundary ξ variables updated:  ✓
  SU(2)/SU(1,1) elements updated: ✓
  SO(1,3) frames corrected:       ✓

  SL(2,C) matrices updated:      ✓
  SL(2,C) parity matrices updated: ✓
  Boundary bivectors updated:    ✓
  boundary ξ variables updated:  ✓
  SU(2)/SU(1,1) elements updated: ✓
  SO(1,3) frames corrected:       ✓

Global connectivity constructed for both relevant orientations.

   FACE–XI GEOMETRY CHECKS

✓ Face matching satisfied (max residual = 8.144890172189112e-16).
✓ SL(2,C) parallel transport satisfied (max residual = 2.7755575615628914e-16).
✓ Closure condition satisfied (max residual = 1.1775693440128312e-16).

------------------------------
✓ All geometric consistency checks PASSED.
------------------------------


   FACE–XI GEOMETRY CHECKS

✓ Face matching satisfied (max residual = 8.1

In [50]:
# ------------------------------------------------------------
# 7a. Symbols and action (reference orientation)
# ------------------------------------------------------------
LorentzianSimplexSolver.DefineSymbols.run_define_variables(geom_ref)

sd_ref, _ = LorentzianSimplexSolver.SolveVars.run_solver(geom_ref)

S_ref = LorentzianSimplexSolver.DefineAction.compute_action(geom_ref)

S_ref_fn, labels_ref =
    LorentzianSimplexSolver.SymbolicToJulia.build_action_function(S_ref, sd_ref)

using Symbolics
@variables γ
args_ref =
    LorentzianSimplexSolver.SymbolicToJulia.build_argument_vector(sd_ref, labels_ref, γ)
args_ref_keep_j =
    LorentzianSimplexSolver.SymbolicToJulia.build_argument_vector_keep_j(sd_ref, labels_ref, γ)

S_ref_sym = expand(simplify(S_ref_fn(args_ref...)))
S_ref_sym_keep_j = expand(simplify(S_ref_fn(args_ref_keep_j...)))

im*(6.13983773827422j_1_2_3 - 6.283185307179586j_1_2_5 + 2.356194490192345j_1_4_5 + 6.283185307179586j_1_5_1 - 6.283185307179586j_2_2_1 - 6.283185307179586j_2_3_5 + 3.8489532953322683j_2_4_1 + 6.283185307179586j_2_5_2 + 2.356194490192345j_2_5_4 - 0.5493061443340551j_1_1_2*γ + 0.7953654612239063j_1_2_3*γ - 0.5493061443340553j_2_3_2*γ)

In [30]:
# ------------------------------------------------------------
# 7b. Symbols and action (parity orientation)
# ------------------------------------------------------------
LorentzianSimplexSolver.DefineSymbols.run_define_variables(geom_parity)

sd_parity, _ = LorentzianSimplexSolver.SolveVars.run_solver(geom_parity)

S_parity = LorentzianSimplexSolver.DefineAction.compute_action(geom_parity)

S_parity_fn, labels_parity =
    LorentzianSimplexSolver.SymbolicToJulia.build_action_function(S_parity, sd_parity)

args_parity =
    LorentzianSimplexSolver.SymbolicToJulia.build_argument_vector(sd_parity, labels_parity, γ)
args_parity_keep_j =
    LorentzianSimplexSolver.SymbolicToJulia.build_argument_vector_keep_j(sd_parity, labels_parity, γ)

S_parity_sym = expand(simplify(S_parity_fn(args_parity...)))
S_parity_sym_keep_j = expand(simplify(S_ref_fn(args_parity_keep_j...)))

(im*(2.220446049250313e-16j_1_5_3 - 1.622186464368052e-32j_1_1_2*γ + 3.1415926535897936j_1_1_4*γ + 6.13983773827422j_1_2_3*γ + 3.1415926535897936j_1_2_5*γ - 3.141592653589793j_1_4_2*γ + 3.926990816987242j_1_4_5*γ - 3.1415926535897936j_1_5_1*γ + 6.283185307179586j_1_5_3*γ - 6.283185307179586j_2_2_1*γ + 3.141592653589793j_2_2_4*γ + 3.1415926535897936j_2_3_5*γ - 4.528627114240512j_2_4_1*γ - 3.141592653589793j_2_4_3*γ - 3.1415926535897936j_2_5_2*γ - 2.356194490192345j_2_5_4*γ + 0.5493061443340552j_1_1_2*(γ^2) - 0.795365461223906j_1_2_3*(γ^2) - 4.440892098500626e-16j_1_2_5*(γ^2) + 3.3306690738754696e-16j_2_2_1*(γ^2) + 0.5493061443340551j_2_3_2*(γ^2) - 4.440892098500626e-16j_2_3_5*(γ^2))) / γ

In [ ]:
function chop_small_coeffs(expr; atol=1e-12)
    # First split the γ-structure
    expr = split_gamma(expr, γ)

    # Work only with the imaginary phase
    F = imag(expr)

    # Collect variables and remove γ safely
    vars = Symbolics.get_variables(F)

    vars = filter(v -> !isequal(v, γ), vars)

    out = zero(F)

    for v in vars
        c = Symbolics.coeff(F, v)
        c_test = Symbolics.substitute(c, Dict(γ => 1))

        cnum = try
            Float64(c_test)
        catch
            # symbolic coefficient → keep
            out += c * v
            continue
        end

        if abs(cnum) ≥ atol
            out += c * v
        end
        
    end
    
end

chop_small_coeffs (generic function with 1 method)

In [138]:
chop_small_coeffs(S_parity_sym_keep_j)

3.141592653589793-3.1415926535897936.283185307179586 + 2.220446049250313e-16 / γ3.1415926535897936 - 4.440892098500626e-16γ-4.528627114240512-1.622186464368052e-32 + 0.5493061443340552γ-2.356194490192345-3.141592653589793-3.14159265358979363.14159265358979366.13983773827422 - 0.795365461223906γ-3.14159265358979363.1415926535897936 - 4.440892098500626e-16γ-6.283185307179586 + 3.3306690738754696e-16γ3.9269908169872420.5493061443340551γ

In [52]:
expr1 = split_gamma(S_ref_sym_keep_j, γ)
expr2 = split_gamma(S_parity_sym_keep_j, γ)

im*(-1.622186464368052e-32j_1_1_2 + 3.1415926535897936j_1_1_4 + 6.13983773827422j_1_2_3 + 3.1415926535897936j_1_2_5 - 3.141592653589793j_1_4_2 + 3.926990816987242j_1_4_5 - 3.1415926535897936j_1_5_1 + 6.283185307179586j_1_5_3 - 6.283185307179586j_2_2_1 + 3.141592653589793j_2_2_4 + 3.1415926535897936j_2_3_5 - 4.528627114240512j_2_4_1 - 3.141592653589793j_2_4_3 - 3.1415926535897936j_2_5_2 - 2.356194490192345j_2_5_4 + (2.220446049250313e-16j_1_5_3) / γ + 0.5493061443340552j_1_1_2*γ - 0.795365461223906j_1_2_3*γ - 4.440892098500626e-16j_1_2_5*γ + 3.3306690738754696e-16j_2_2_1*γ + 0.5493061443340551j_2_3_2*γ - 4.440892098500626e-16j_2_3_5*γ)

In [93]:
phase = split_gamma(expand(simplify((expr1 + expr2)//2)), γ)

im*(-0.27465307216702756j_1_1_2 + 1.5707963267948968j_1_1_4 + 3.4676015997490635j_1_2_3 + 1.5707963267948968j_1_2_5 - 1.5707963267948966j_1_4_2 + 1.963495408493621j_1_4_5 - 1.5707963267948968j_1_5_1 + 3.141592653589793j_1_5_3 - 3.141592653589793j_2_2_1 + 1.5707963267948966j_2_2_4 - 0.27465307216702767j_2_3_2 + 1.5707963267948968j_2_3_5 - 2.264313557120256j_2_4_1 - 1.5707963267948966j_2_4_3 - 1.5707963267948968j_2_5_2 - 1.1780972450961724j_2_5_4 + (3.06991886913711j_1_2_3 - 3.141592653589793j_1_2_5 + 1.1780972450961724j_1_4_5 + 3.141592653589793j_1_5_1 - 3.141592653589793j_2_2_1 - 3.141592653589793j_2_3_5 + 1.9244766476661341j_2_4_1 + 3.141592653589793j_2_5_2 + 1.1780972450961724j_2_5_4) / γ + 0.2746530721670276j_1_1_2*γ - 0.397682730611953j_1_2_3*γ + 0.27465307216702756j_2_3_2*γ)

In [89]:
expr_ref = split_gamma(expand(simplify(expr1 - phase)), γ)
expr_parity = split_gamma(expand(simplify(expr2 - phase)), γ)

im*(0.27465307216702756j_1_1_2 + 1.5707963267948968j_1_1_4 + 2.672236138525157j_1_2_3 + 1.5707963267948968j_1_2_5 - 1.5707963267948966j_1_4_2 + 1.963495408493621j_1_4_5 - 1.5707963267948968j_1_5_1 + 3.141592653589793j_1_5_3 - 3.141592653589793j_2_2_1 + 1.5707963267948966j_2_2_4 + 0.27465307216702767j_2_3_2 + 1.5707963267948968j_2_3_5 - 2.264313557120256j_2_4_1 - 1.5707963267948966j_2_4_3 - 1.5707963267948968j_2_5_2 - 1.1780972450961724j_2_5_4 + (-3.06991886913711j_1_2_3 + 3.141592653589793j_1_2_5 - 1.1780972450961724j_1_4_5 - 3.141592653589793j_1_5_1 + 3.141592653589793j_2_2_1 + 3.141592653589793j_2_3_5 - 1.9244766476661341j_2_4_1 - 3.141592653589793j_2_5_2 - 1.1780972450961724j_2_5_4) / γ + 0.2746530721670276j_1_1_2*γ - 0.397682730611953j_1_2_3*γ + 0.27465307216702756j_2_3_2*γ)

In [94]:
S_regge_symbolics

-0.7853981633974483j_1_4_5 - 3.141592653589793j_1_5_3 - 2.0943951023931957j_2_4_1 - 0.7853981633974483j_2_5_4 + 0.5493061443340551j_1_1_2*γ - 0.7953654612239056j_1_2_3*γ + 0.5493061443340551j_2_3_2*γ

In [64]:
S_regge_num,  S_regge_symbolics = LorentzianSimplexSolver.ReggeAction.run_Regge_action(geom_ref, γ);

In [83]:
exprx = split_gamma(expand(simplify(expr_ref + im * S_regge_symbolics)), γ)

im*(-0.27465307216702756j_1_1_2 - 1.5707963267948968j_1_1_4 - 2.6722361385251574j_1_2_3 - 1.5707963267948968j_1_2_5 + 1.5707963267948966j_1_4_2 - 2.7488935718910694j_1_4_5 + 1.5707963267948968j_1_5_1 - 6.283185307179586j_1_5_3 + 3.141592653589793j_2_2_1 - 1.5707963267948966j_2_2_4 - 0.27465307216702767j_2_3_2 - 1.5707963267948968j_2_3_5 + 0.16991845472706046j_2_4_1 + 1.5707963267948966j_2_4_3 + 1.5707963267948968j_2_5_2 + 0.39269908169872414j_2_5_4 + (3.06991886913711j_1_2_3 - 3.141592653589793j_1_2_5 + 1.1780972450961724j_1_4_5 + 3.141592653589793j_1_5_1 - 3.141592653589793j_2_2_1 - 3.141592653589793j_2_3_5 + 1.9244766476661341j_2_4_1 + 3.141592653589793j_2_5_2 + 1.1780972450961724j_2_5_4) / γ + 0.2746530721670275j_1_1_2*γ - 0.39768273061195264j_1_2_3*γ + 0.27465307216702756j_2_3_2*γ)

In [86]:
function check_pi_over_2_coeffs(expr; tol=1e-6)
    expr = Symbolics.expand(expr)

    vars = Symbolics.get_variables(expr)

end

check_pi_over_2_coeffs (generic function with 1 method)

In [87]:
check_pi_over_2_coeffs(exprx)

SymbolicUtils.BasicSymbolic[]

In [80]:
function split_gamma(S::Complex{Num}, γ::Num)
    # scalar phase
    F = Symbolics.expand(Symbolics.imag(S))

    num = Symbolics.expand(Symbolics.numerator(F))
    # den = Symbolics.denominator(F)   # not needed, known = γ

    # extract coefficients
    c2 = Symbolics.coeff(num, γ^2)
    c1 = Symbolics.coeff(num, γ)
    c0 = Symbolics.expand(num - c1*γ - c2*γ^2)

    return im * Symbolics.expand(c0/γ + c1 + c2*γ)
end

function check_pi_over_2_coeffs(expr; tol=1e-6)
    expr = Symbolics.expand(expr)

    vars = Symbolics.get_variables(expr)

    println("Checking coefficients (in units of π/2):\n")

    for v in vars
        c = Symbolics.coeff(expr, v)
        c_val = Symbolics.simplify(c / (pi/2))

        # try numeric evaluation
        c_num = try
            Float64(c_val)
        catch
            continue
        end

        nearest = round(c_num)
        ok = abs(c_num - nearest) < tol

        println(rpad(string(v), 15),
                "  coeff = ", c,
                "  -> ", c_num,
                ok ? "  ✓ integer" : "  ✗ NOT integer")
    end
end

function select_orientation(
    S_ref::Complex{Num},
    S_parity::Complex{Num},
    S_regge::Num,
    γ::Num
)
    # average phase
    phase = Symbolics.expand((S_ref + S_parity) // 2)

    # # differences
    Δref_minus = Symbolics.simplify(S_ref - phase + im*S_regge; expand=true)
    split_gamma(Δref_minus, γ)

end

select_orientation (generic function with 1 method)

In [125]:
S_ref_sym_keep_j

im*(6.13983773827422j_1_2_3 - 6.283185307179586j_1_2_5 + 2.356194490192345j_1_4_5 + 6.283185307179586j_1_5_1 - 6.283185307179586j_2_2_1 - 6.283185307179586j_2_3_5 + 3.8489532953322683j_2_4_1 + 6.283185307179586j_2_5_2 + 2.356194490192345j_2_5_4 - 0.5493061443340551j_1_1_2*γ + 0.7953654612239063j_1_2_3*γ - 0.5493061443340553j_2_3_2*γ)

In [124]:
select_orientation(S_ref_sym_keep_j, S_parity_sym_keep_j, S_regge_symbolics, γ)

im*(8.11093232184026e-33j_1_1_2 - 1.5707963267948968j_1_1_4 - 4.71238898038469j_1_2_5 + 1.5707963267948966j_1_4_2 - 1.5707963267948968j_1_4_5 + 4.71238898038469j_1_5_1 - 6.283185307179586j_1_5_3 - 1.5707963267948966j_2_2_4 - 4.71238898038469j_2_3_5 + 2.094395102393195j_2_4_1 + 1.5707963267948966j_2_4_3 + 4.71238898038469j_2_5_2 + 1.5707963267948966j_2_5_4 + (-1.1102230246251565e-16j_1_5_3) / γ - 5.551115123125783e-17j_1_1_2*γ + 4.996003610813204e-16j_1_2_3*γ + 2.220446049250313e-16j_1_2_5*γ - 1.6653345369377348e-16j_2_2_1*γ - 1.1102230246251565e-16j_2_3_2*γ + 2.220446049250313e-16j_2_3_5*γ)

In [ ]:

    Δpar_minus = Symbolics.simplify(S_parity - phase - im*S_regge; expand=true)


im*(1.5707963267948968j_1_1_4 + 4.71238898038469j_1_2_5 - 1.5707963267948966j_1_4_2 + 1.5707963267948968j_1_4_5 - 4.71238898038469j_1_5_1 + 6.283185307179586j_1_5_3 + 1.5707963267948966j_2_2_4 + 4.71238898038469j_2_3_5 - 2.094395102393195j_2_4_1 - 1.5707963267948966j_2_4_3 - 4.71238898038469j_2_5_2 - 1.5707963267948966j_2_5_4)

In [ ]:

    # decide orientation
    if Symbolics.simplify(Δref_minus; atol=1e-10) == 0
        S_pos = S_ref
        S_neg = S_parity
    else
        S_pos = S_parity
        S_neg = S_ref
    end

    return S_pos, S_neg, phase

In [ ]:
# ------------------------------------------------------------
# 8. EOMs
# ------------------------------------------------------------
dS = LorentzianSimplexSolver.EOMsHessian.compute_EOMs(S, sd)

grad_fns = LorentzianSimplexSolver.SymbolicToJulia.build_gradient_functions(dS, sd)

LorentzianSimplexSolver.EOMsHessian.check_EOMs(grad_fns, sd; γ = one(ScalarT));

LoadError: InterruptException:

In [ ]:
# ------------------------------------------------------------
# 9. Hessian computations
# ------------------------------------------------------------
H = LorentzianSimplexSolver.EOMsHessian.compute_Hessian(S, sd)

hess_fns = LorentzianSimplexSolver.SymbolicToJulia.build_hessian_functions(H, sd)
H_eval, _ = LorentzianSimplexSolver.EOMsHessian.evaluate_hessian(hess_fns, sd; γ = one(ScalarT))

H_det = det(H_eval)